In [1]:
import psycopg2
from psycopg2.extras import execute_values, execute_batch
size = 10000

In [2]:
def gen_conn():
    try:
        conn = psycopg2.connect("dbname='postgres' user='postgres' host='postgres' password='postgres'")
    except:
        print("I am unable to connect to the database")
    return conn


In [3]:
def create_tables(conn):
    commands = [
        """
        CREATE TABLE loop (
            id INT PRIMARY KEY,
            data TEXT
        )
        """,
        """
        CREATE TABLE block (
            id INT PRIMARY KEY,
            data TEXT
        )
        """
    ]
    cur = conn.cursor()
    for command in commands:
        # print(command)
        cur.execute(command)
    cur.close()
    conn.commit()
    
def drop_tables(conn):
    cur = conn.cursor()
    cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
    for table in cur.fetchall():
        print(table[0])
        # Form the SQL statement - DROP TABLE
        dropTableStmt   = "DROP TABLE %s;"%table[0]
        print(f'dropping - {dropTableStmt}')
        cur.execute(dropTableStmt)
    conn.commit()
    

In [4]:
# set up tables

conn=gen_conn()
create_tables(conn)
conn.commit()
conn.close()

In [5]:
%%time
# prep data

data = []
for i in range(size):
    data.append((i,"test"))
print(len(data))


10000
CPU times: user 1.67 ms, sys: 371 µs, total: 2.04 ms
Wall time: 1.97 ms


In [6]:
%%time
# writes in a loop

conn = gen_conn()
sql = """INSERT INTO loop(id,data) VALUES(%s,%s)"""
cur = conn.cursor()
for i in data:
    try:
        cur.execute(sql,(i[0],i[1]))
        conn.commit()
    except:
        conn.rollback() 
conn.close()

CPU times: user 443 ms, sys: 1.3 s, total: 1.74 s
Wall time: 40.1 s


In [7]:
%%time
# writes in a block

conn = gen_conn()
sql = """INSERT INTO block (id,data) VALUES %s""" 
cur = conn.cursor()
execute_values(cur, sql, data, template=None, page_size=100)
conn.commit()
conn.close()

CPU times: user 22.5 ms, sys: 0 ns, total: 22.5 ms
Wall time: 48.3 ms


In [8]:
# prove there is data in loop

conn = gen_conn()
cur=conn.cursor()
cur.execute("""select * from loop""")
print(cur.fetchall()[:10])
conn.close()

[(0, 'test'), (1, 'test'), (2, 'test'), (3, 'test'), (4, 'test'), (5, 'test'), (6, 'test'), (7, 'test'), (8, 'test'), (9, 'test')]


In [9]:
# prove there is data in block

conn = gen_conn()
cur=conn.cursor()
cur.execute("""select * from block""")
print(cur.fetchall()[:10])
# for table in cursor.fetchall():
#     print(table)
conn.close()


[(0, 'test'), (1, 'test'), (2, 'test'), (3, 'test'), (4, 'test'), (5, 'test'), (6, 'test'), (7, 'test'), (8, 'test'), (9, 'test')]


In [10]:
# drop tables

conn=gen_conn()
drop_tables(conn)
conn.commit()
conn.close()

loop
dropping - DROP TABLE loop;
block
dropping - DROP TABLE block;
